In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,stddev
import custom_pckgs.webscrapingFuncs as _scraping

In [2]:
#Link in format link = "https://xyz.cloudfront.net/trip-data/yellow_tripdata_YYYY-MM.parquet"
def get_date_from_link(link): 
    date_str = link.split('_')[-1].split('.')[0]#Gets YYYY-MM from the link
    split_date = date_str.split('-')
    year = split_date[0]
    month = split_date[1]
    year_month = int(year + month) #Integer date in format YYYYMM
    return year_month

In [3]:
def validate_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m")####YYYY-MM Format expected
    except Exception as e:
        raise Exception(f"Invalid date format. Expected format is YYYY-MM.")

In [4]:
def download_taxi_files(start_date, end_date):
    validate_date(start_date)
    validate_date(end_date)

    base_url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
    yellow_taxi_links = []	
    
    start_date_int = int(start_date.replace('-', '')) #Converts to YYYYMM integer
    end_date_int = int(end_date.replace('-', '')) #Converts to YYYYMM integer
    
    page = _scraping.page_requester(base_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    file_table = soup.find('div', class_='faq-v1')# Gets table where all files are listed
    yearly_files=file_table.find_all('div', class_='faq-answers') #Gets all files year by year
    for year in yearly_files:
        a_tags = year.find_all('a', href=True)
        for a_tag in a_tags:
            link = a_tag['href'] #Filters file links for yellow taxis between the date interval
            if 'yellow_tripdata' in link and (start_date_int <= get_date_from_link(link) <= end_date_int):
                clean_link = link.strip()#Some links have trailing spaces
                _scraping.download_file(clean_link)
                yellow_taxi_links.append(clean_link)
    return yellow_taxi_links

In [5]:
yellow_taxi_links = download_taxi_files('2024-10', '2024-12')

In [6]:
#Initiating SparkSession and dataset_path
spark = SparkSession.builder.appName("SparkExercise").getOrCreate()
dataset_path = "./dataset/"

In [7]:
df = spark.read.parquet(dataset_path)

Data exploration

In [8]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [9]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

Top 10 % trips based on trip_distance

In [10]:
row_count = df.count()

top10_percent_count = int(row_count * 0.1)

sorted_df = df.orderBy("trip_distance", ascending=False)

#Get top 10% of trips based on trip distance
top10_percent_df = sorted_df.limit(top10_percent_count)

In [11]:
top10_percent_df.show() #Sample

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-11-28 19:28:00|  2024-11-28 19:42:00|           null|    398608.62|      null|              null|         151|         140|           0|      22.04|  0.0|    0.5|       0.

In [12]:
#Stats for total_amount column
top10_percent_df.describe('total_amount').show()

+-------+------------------+
|summary|      total_amount|
+-------+------------------+
|  count|            748014|
|   mean| 78.74998038813133|
| stddev|36.389930487574134|
|    min|          -1477.68|
|    max|           1690.89|
+-------+------------------+



In [13]:
top10_percent_df.filter(top10_percent_df["total_amount"].isNull()).count() #No null values -> 0

0

In [14]:
top10_percent_df.filter(top10_percent_df["total_amount"] < 0).count() #Negative and 0 values present, might be due to refunds, promotional discounts or data errors

16014

Data Quality & Cleaning

In [15]:
# 0 or negative values will be filtered out as there is not any specification on the dataset about the meaning of these values
positive_total_df = top10_percent_df.filter(top10_percent_df["total_amount"] > 0)

In [16]:
positive_total_df.describe('total_amount').show() #Stats 
#Total amount might be affected buy outliers as the maximum value is way higher than the avg, so we could filter out if needed.

+-------+------------------+
|summary|      total_amount|
+-------+------------------+
|  count|            731960|
|   mean| 82.19611481508397|
| stddev|27.696012870407383|
|    min|              0.01|
|    max|           1690.89|
+-------+------------------+



###### Next block is OPTIONAL because high amount might be reasonable with the trip distance(but trip distance values are really high so they could be incorrect)

In [17]:
avg = positive_total_df.agg(mean("total_amount"))
sttdev = positive_total_df.agg(stddev("total_amount"))

In [18]:
#Filter outliers based on avg and standard deviation, in this case lower interval does not filter values as we had removed negative values
cleaned_df = positive_total_df.filter(
    (positive_total_df["total_amount"] >= avg.collect()[0][0] - 3 * sttdev.collect()[0][0]) &
    (positive_total_df["total_amount"] <= avg.collect()[0][0] + 3 * sttdev.collect()[0][0])
)

Final result

In [19]:
cleaned_df.describe('total_amount').show()

+-------+-----------------+
|summary|     total_amount|
+-------+-----------------+
|  count|           723423|
|   mean|80.48891591786648|
| stddev|21.49635956018364|
|    min|             0.01|
|    max|           165.25|
+-------+-----------------+



In [ ]:
cleaned_df.show()